In [ ]:
!python3 -m spacy download en_core_web_sm

import spacy

nlp = spacy.load('en_core_web_sm', disable=['ner'])

In [ ]:
def convert_list_to_string(list):
    str = ""
    for l in list:
        str = str.replace("\n", " ") + l
    return str

In [ ]:
agreement1 = convert_list_to_string(open("aws_ec2_sla_may_2022.txt").readlines())
agreement2 = convert_list_to_string(open("google_engine_sla.txt").readlines())
agreement3 = convert_list_to_string(open("oracle_cloud_infrastructure_database_december_2022.txt").readlines())
agreement4 = convert_list_to_string(open("twilio_sla_august_2022.txt").readlines())
agreement5 = convert_list_to_string(open("microsoft_azure_kubernetes_sla_march_2020.txt").readlines())
agreement6 = convert_list_to_string(open("ibm_sla_september_2022.txt").readlines())
agreement7 = convert_list_to_string(open("aws_glue_sla_may_2022.txt").readlines())
agreement8 = convert_list_to_string(open("aws_cds_sla_may_2022.txt").readlines())
agreement2

In [ ]:
print("Procesando con Spacy...")
textos_train = [agreement1, agreement2, agreement3, agreement4]
textos_test = [agreement5, agreement7]
docs_train = list(nlp.pipe(textos_train))
docs_test = list(nlp.pipe(textos_test))
print("Hecho.")

In [ ]:
sents_train = []

for i in range(len(docs_train)):
  for sent in list(docs_train[i].sents):
      sentence = str(sent)
      sents_train.append(sentence)

In [ ]:
sents_test = []
for sent in list(docs_test[0].sents):
    sentence = str(sent)
    sents_test.append(sentence)

In [ ]:
import pandas as pd

df = pd.DataFrame(sents_train, columns=["text", ])
df["service"] = 0
df["metric"] = 0
df["objetive"] = 0
df["remedies"] = 0
df["claim"] = 0
df["exception"] = 0
df["definition"] = 0
df.fillna(0, inplace=True)
df.to_csv("train.csv", index=False, encoding='utf-8')
print(df.head())

In [ ]:
import pandas as pd

df = pd.DataFrame(sents_test, columns=["text", ])
df["service"] = 0
df["metric"] = 0
df["objetive"] = 0
df["remedies"] = 0
df["claim"] = 0
df["exception"] = 0
df["definition"] = 0
df.fillna(0, inplace=True)
df.to_csv("validation.csv", index=False, encoding='utf-8')
print(df.head())